# Analogous stats

In [ ]:
using Pkg
Pkg.activate("../../env/.")

In [1]:
include("../../src/analogous_bars.jl")
include("../../src/Eirene_var.jl")
include("../../src/eirene_helper.jl")
using .analogous_bars
using .Eirene_var
using .eirene_helper
using DelimitedFiles
using HDF5
using Plots
using FileIO
using Random


WebIO._IJuliaInit()

┌ Warning: ORCA.jl has been deprecated and all savefig functionality
│ has been implemented directly in PlotlyBase itself.
│ 
│ By implementing in PlotlyBase.jl, the savefig routines are automatically
│ available to PlotlyJS.jl also.
└ @ ORCA /opt/julia/packages/ORCA/U5XaN/src/ORCA.jl:8


In [2]:
using Distributed
#addprocs(5);

In [3]:
nprocs()

1

In [4]:
@everywhere include("../../../src/analogous_stats.jl")
@everywhere using .analogous_stats

## 1. conjunctive & grid cells

Load dissimilarity matrices

In [5]:
directory = "simulated_data/"

n_grid = 200
n_conj = 800

D_grid = h5read(directory * "grid_dissimilarity.h5", "distance")
D_grid = vector_to_symmetric_matrix(D_grid, n_grid)

D_conj = h5read(directory * "conj_dissimilarity.h5", "distance")
D_conj = vector_to_symmetric_matrix(D_conj, n_conj)

D_grid_conj = h5read(directory * "conj_grid_rate_dissimilarity.h5", "distance")
D_conj_grid = Array(transpose(D_grid_conj));

Compute VR persistence

In [6]:
VR_conj = eirene(D_conj, record = "all", maxdim = 1)
VR_grid = eirene(D_grid, record = "all", maxdim = 1);

LoadError: [91mInterruptException:[39m

Load the VR, W persistence diagrams and cutoffs

In [7]:
outputs = load("simulated_data/grid_conj_analogous_outputs.jld2")

# load barcodes
barcode_conj = outputs["barcodes"]["VR_conj"]
barcode_grid = outputs["barcodes"]["VR_grid"]
barcode_W_grid_conj = outputs["barcodes"]["W_grid_conj"]

# load cutoffs
conj_cutoff = outputs["persistence_cutoff"]["conj_cutoff"]
grid_cutoff = outputs["persistence_cutoff"]["grid_cutoff"]
W_cutoff = outputs["persistence_cutoff"]["W_grid_conj_cutoff"];

In [8]:
selected_conj = findall(x -> x > conj_cutoff, barcode_conj[:,2] - barcode_conj[:,1])
selected_grid = findall(x -> x > grid_cutoff, barcode_grid[:,2] - barcode_grid[:,1])

2-element Array{Int64,1}:
 58
 59

### Compute using distributed computation

In [ ]:
# script to compute from 50 to 100
start_idx = collect(101:5:200)
for idx in start_idx
    shuffles = collect(idx:idx+4) 
    t = @distributed for k in shuffles
        output_filename = "null_model_stats/grid_conj/" * string(k)
        _, _, _, _, _ = analogous_stats.compute_analogous_bars_null_model(;
                                                                    VR_P = VR_grid,
                                                                    D_P = D_grid,
                                                                    VR_Q = VR_conj,
                                                                    D_Q = D_conj,
                                                                    P_null_dim = 2,
                                                                    Q_null_dim = 3,
                                                                    output_filename = output_filename
                                                                    )
    

    end
    fetch(t)
end

## load null model stats

In [10]:
n_shuffles = 200
output_dir = "null_model_stats/grid_conj/"
selected_grid = [58, 59]
selected_conj = [435, 436, 437]
stats_matrix, stats_matrix_total, selected_bar_count, no_Ftau_P_or_Q, valid_files = parse_null_model_stats(output_dir, 
                                                                         barcode_grid, 
                                                                         barcode_conj,
                                                                         selected_grid,
                                                                         selected_conj,
                                                                         n_shuffles = n_shuffles)

prob_matrix = stats_matrix ./ n_shuffles

2×3 Array{Float64,2}:
 0.0  0.0  0.0
 0.0  0.0  0.0

In [13]:
maximum(stats_matrix_total)

1.0

In [23]:
1/200

0.005

In [22]:
1/170

0.0058823529411764705

In [21]:
# count the total number of bars from which we ran similarity_centric_analogous_bars
output_dir = "null_model_stats/grid_conj/"
for i = 1:10


    # open julia files
    analogous_P = load(output_dir * string(i) * ".jld2")["analogous_P"]
    analogous_Q = load(output_dir * string(i) * ".jld2")["analogous_Q"]
    selected = load(output_dir * string(i) * ".jld2")["W_selected"]

    println(selected)
    println(analogous_P)
    println(analogous_Q)
end

[48, 50]
Dict{Any,Any}(50 => [1],48 => [8, 1])
Dict{Any,Any}(50 => nothing,48 => nothing)
[44, 45]
Dict{Any,Any}(44 => nothing,45 => nothing)
Dict{Any,Any}(44 => nothing,45 => nothing)
[55, 56]
Dict{Any,Any}(55 => nothing,56 => nothing)
Dict{Any,Any}(55 => nothing,56 => nothing)
[55, 56]
Dict{Any,Any}(55 => nothing,56 => nothing)
Dict{Any,Any}(55 => nothing,56 => nothing)
[44, 46]
Dict{Any,Any}(44 => nothing,46 => nothing)
Dict{Any,Any}(44 => nothing,46 => nothing)
[54, 55]
Dict{Any,Any}(54 => nothing,55 => [3])
Dict{Any,Any}(54 => nothing,55 => nothing)
[49, 51]
Dict{Any,Any}(49 => [8],51 => nothing)
Dict{Any,Any}(49 => nothing,51 => nothing)
[42, 43]
Dict{Any,Any}(43 => nothing,42 => nothing)
Dict{Any,Any}(43 => nothing,42 => nothing)
[44, 47]
Dict{Any,Any}(47 => [2],44 => nothing)
Dict{Any,Any}(47 => nothing,44 => nothing)
[46, 47]
Dict{Any,Any}(47 => nothing,46 => nothing)
Dict{Any,Any}(47 => nothing,46 => nothing)


# 2. HD cells & conj cells

Load dissimilarity matrices

In [14]:
directory = "simulated_data/"

n_HD = 64
n_conj = 800

D_HD = h5read(directory * "HD_dissimilarity.h5", "distance")
D_HD = vector_to_symmetric_matrix(D_HD, n_HD)

D_conj = h5read(directory * "conj_dissimilarity.h5", "distance")
D_conj = vector_to_symmetric_matrix(D_conj, n_conj)

D_HD_conj = h5read(directory * "conj_HD_rate_dissimilarity.h5", "distance")
D_conj_HD = Array(transpose(D_HD_conj));

Compute VR persistence

In [13]:
VR_conj = eirene(D_conj, record = "all", maxdim = 1)
VR_HD = eirene(D_HD, record = "all", maxdim = 1);


Load the VR, W persistence diagrams and cutoffs

In [15]:
outputs = load("simulated_data/HD_conj_analogous_outputs.jld2")

# load barcodes
barcode_conj = outputs["barcodes"]["VR_conj"]
barcode_HD = outputs["barcodes"]["VR_HD"]
barcode_W_HD_conj = outputs["barcodes"]["W_HD_conj"]

# load cutoffs
conj_cutoff = outputs["persistence_cutoff"]["conj_cutoff"]
HD_cutoff = outputs["persistence_cutoff"]["HD_cutoff"]
W_cutoff = outputs["persistence_cutoff"]["W_HD_conj_cutoff"]

# selected bars
selected_conj = findall(x -> x > conj_cutoff, barcode_conj[:,2] - barcode_conj[:,1])
selected_HD = findall(x -> x > HD_cutoff, barcode_HD[:,2] - barcode_HD[:,1]);

In [ ]:
# script to compute from 50 to 100
start_idx = collect(11:5:100)
for idx in start_idx
    shuffles = collect(idx:idx+4) 
    t = @distributed for k in shuffles
        output_filename = "null_model_stats/HD_conj/" * string(k)
        _, _, _, _, _ = analogous_stats.compute_analogous_bars_null_model(;
                                                                    VR_P = VR_HD,
                                                                    D_P = D_HD,
                                                                    VR_Q = VR_conj,
                                                                    D_Q = D_conj,
                                                                    P_null_dim = 1,
                                                                    Q_null_dim = 3,
                                                                    output_filename = output_filename
                                                                    )
    end
    fetch(t)
end

## Load null model stats

In [19]:
n_shuffles = 100
output_dir = "null_model_stats/HD_conj/"

stats_matrix, stats_matrix_total, selected_bar_count, no_Ftau_P_or_Q, valid_files = parse_null_model_stats(output_dir, 
                                                                         barcode_HD, 
                                                                         barcode_conj,
                                                                         selected_HD,
                                                                         selected_conj,
                                                                         n_shuffles = n_shuffles)

prob_matrix = stats_matrix ./ n_shuffles

1×3 Array{Float64,2}:
 0.0  0.0  0.0

In [21]:
maximum(stats_matrix_total)

0.0

In [19]:
prob_matrix

1×3 Array{Float64,2}:
 0.0  0.0  0.0

In [24]:
1/200

0.005